In [13]:
# Imports e paths

from pathlib import Path
import re
import pandas as pd
import numpy as np
from datetime import datetime
import random
import os

# Paths relativos ao local deste notebook: notebooks/bertopic/01_prep_light.ipynb
BASE = Path("../../data")
RAW = BASE / "raw"
OUT_DIR = BASE / "interim" / "bertopic"
IN_CSV = RAW / "tccs.csv"
OUT_CSV = OUT_DIR / "prep.csv"

# Reprodutibilidade
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# Garantir diretório de saída
OUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"[{datetime.now()}] BASE={BASE.resolve()}")
print(f"IN_CSV={IN_CSV.resolve()}")
print(f"OUT_CSV={OUT_CSV.resolve()}")

[2025-09-08 13:57:24.716580] BASE=C:\Users\User\Desktop\TCC\Notebooks locais\analise_topicos_tcc\data
IN_CSV=C:\Users\User\Desktop\TCC\Notebooks locais\analise_topicos_tcc\data\raw\tccs.csv
OUT_CSV=C:\Users\User\Desktop\TCC\Notebooks locais\analise_topicos_tcc\data\interim\bertopic\prep.csv


In [14]:
# Leitura e chaves

# Leitura
df = pd.read_csv(IN_CSV, encoding="utf-8")

# Validações mínimas
assert "resumo" in df.columns, "A coluna 'resumo' não foi encontrada em data/raw/tccs.csv"

# Garante DOC_ID sequencial se não existir
if "DOC_ID" not in df.columns:
    df.insert(0, "DOC_ID", range(len(df)))

# Snapshot inicial
print(f"Shape: {df.shape}")
print("Colunas:", list(df.columns)[:10], "...")
display(df.head(3))

Shape: (423, 7)
Colunas: ['DOC_ID', 'ano', 'titulo', 'autor', 'orientador', 'resumo', 'url'] ...


,DOC_ID,ano,titulo,autor,orientador,resumo,url
0,0,2024,AGENDEVC: um sistema de agendamento para prest...,"FERREIRA, Williamberg de Albuquerque.","MASSONI, Tiago Lima.",Gerenciar a agenda e fornecer informações ao c...,http://dspace.sti.ufcg.edu.br:8080/jspui/handl...
1,1,2024,Análise de técnicas de explicabilidade em rede...,"SILVA, Wendson Magalhães da.","GOMES, Herman Martins.","Doenças oftalmológicas, como catarata, glaucom...",http://dspace.sti.ufcg.edu.br:8080/jspui/handl...
2,2,2024,O impacto do uso de tags de rastreamento na pe...,"RIBEIRO, Vinicius Trindade Rocha.","MONGIOVI, Melina.",O presente trabalho busca compreender se a uti...,http://dspace.sti.ufcg.edu.br:8080/jspui/handl...


In [15]:
# Função clean_text_light

# Compilações de regex (URLs, e-mails, números isolados)
RE_URL = re.compile(r"https?://\S+|www\.\S+", flags=re.IGNORECASE)
RE_EMAIL = re.compile(r"\b\S+@\S+\b", flags=re.IGNORECASE)
RE_NUM_ISOLATED = re.compile(r"\b\d+\b")
RE_SPACES = re.compile(r"\s+")

def clean_text_light(s: str) -> str:
    """
    Pré-processamento leve para BERTopic:
      - lower() mantendo acentos;
      - remove URLs e e-mails;
      - remove números isolados (não remove tokens alfanuméricos);
      - normaliza espaços;
      - NÃO remove stopwords aqui; NÃO lematiza; NÃO aplica unidecode;
      - mantém pontuação/acentos (útil ao modelo multilíngue).
    """
    s = s.lower()
    s = RE_URL.sub(" ", s)
    s = RE_EMAIL.sub(" ", s)
    s = RE_NUM_ISOLATED.sub(" ", s)
    s = RE_SPACES.sub(" ", s).strip()
    return s

In [16]:
# Aplicação e diagnósticos

# Garante dtype string e trata NaN
orig_resumo = df["resumo"].copy()
df["RESUMO_PREP_BERTOPIC"] = (
    df["resumo"].fillna("").astype(str).map(clean_text_light)
)

# Diagnósticos simples (não filtrantes)
n_total = len(df)
n_vazios_antes = (orig_resumo.fillna("").astype(str).str.strip() == "").sum()
n_vazios_depois = (df["RESUMO_PREP_BERTOPIC"].str.strip() == "").sum()

print(f"Documentos totais: {n_total}")
print(f"Resumos vazios (antes): {n_vazios_antes}")
print(f"Resumos vazios (depois): {n_vazios_depois}")

# Pequeno check de mudanças (amostra)
sample_idx = df.index[:3].tolist()
for i in sample_idx:
    print("\n--- DOC_ID", df.loc[i, "DOC_ID"], "---")
    print("[ANTES]", str(orig_resumo.loc[i])[:300])
    print("[DEPOIS]", df.loc[i, "RESUMO_PREP_BERTOPIC"][:300])

Documentos totais: 423
Resumos vazios (antes): 14
Resumos vazios (depois): 14

--- DOC_ID 0 ---
[ANTES] Gerenciar a agenda e fornecer informações ao cliente são tarefas que oneram muito o tempo do pequeno empreendedor, que, em muitos casos, gerencia e opera um negócio sozinho, precisando maximizar seu tempo produzindo para obter melhores resultados e manter o seu empreendimento em atividade. Este trab
[DEPOIS] gerenciar a agenda e fornecer informações ao cliente são tarefas que oneram muito o tempo do pequeno empreendedor, que, em muitos casos, gerencia e opera um negócio sozinho, precisando maximizar seu tempo produzindo para obter melhores resultados e manter o seu empreendimento em atividade. este trab

--- DOC_ID 1 ---
[ANTES] Doenças oftalmológicas, como catarata, glaucoma e retinopatia diabética, representam um desafio significativo para a saúde pública, com potencial de causar perda de visão. No entanto, a maioria desses casos poderia ser evitada ou tratada se diagnosticada prec

In [ ]:
# === PATCH: re-gerar Fig. 3, 5, 6 e 7 sem alterar a pipeline ===
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
from textwrap import shorten
import re

OUT_QC = Path("../../data/interim/quality/bertopic")
OUT_QC.mkdir(parents=True, exist_ok=True)

def plot_hist_safe(arr_like, title, xlabel, out_path):
    s = pd.Series(arr_like, dtype="float")
    s = pd.to_numeric(s, errors="coerce").dropna()
    fig, ax = plt.subplots(figsize=(7,4))
    ax.hist(s, bins=50, edgecolor="black")
    med = s.median()
    q1, q3 = s.quantile([0.25, 0.75])
    ax.axvline(med, linestyle="--", linewidth=1.5, label=f"Mediana = {med:.1f}")
    ax.set_title(title)
    ax.set_xlabel(xlabel); ax.set_ylabel("Frequência")
    ax.legend()
    ax.text(0.98, 0.95, f"N={len(s)}\nIQR≈{(q3-q1):.1f}", transform=ax.transAxes,
            ha="right", va="top", fontsize=9, bbox=dict(facecolor="white", alpha=0.7, edgecolor="none"))
    fig.tight_layout(); fig.savefig(out_path, dpi=200); plt.close(fig)

# Fig. 3
plot_hist_safe(n_tokens, "Comprimento de documentos (tokens) pós-limpeza", "nº de tokens",
               OUT_QC / "fig03_doc_length_tokens_bertopic.png")
plot_hist_safe(n_chars, "Comprimento de documentos (caracteres) pós-limpeza", "nº de caracteres",
               OUT_QC / "fig03_doc_length_chars_bertopic.png")

# Fig. 5
plot_hist_safe(nonalpha_ratio, "Proporção de tokens não alfabéticos por documento após pré-processamento de BERTopic", "proporção (0–1)",
               OUT_QC / "fig05_nonalpha_ratio_bertopic.png")

# Fig. 6
plot_hist_safe(stopw_ratio, "Proporção de stopwords por documento (sem remoção prévia)", "proporção (0–1)",
               OUT_QC / "fig06_stopwords_ratio_bertopic.png")

# Fig. 7 — tabela de exemplos (reaproveita RE_URL/RE_EMAIL/RE_NUM_ISOLATED/RE_SPACES)
def excerpt(s, width=160):
    try:
        return shorten(" ".join(str(s).split()), width=width, placeholder="…")
    except Exception:
        return str(s)[:width]

def flags_acionadas(s: str):
    if not isinstance(s, str): s = ""
    hits = []
    try:
        if RE_URL.search(s):            hits.append("URL")
    except Exception: pass
    try:
        if RE_EMAIL.search(s):          hits.append("email")
    except Exception: pass
    try:
        if RE_NUM_ISOLATED.search(s):   hits.append("número isolado")
    except Exception: pass
    try:
        if RE_SPACES.search(s):         hits.append("múltiplos espaços")
    except Exception: pass
    return ", ".join(hits) if hits else "—"

ex = pd.DataFrame({"raw": texts_raw, "clean": texts_after})
ex["regras_acionadas"] = [flags_acionadas(x) for x in texts_raw]
cand = ex[(ex["regras_acionadas"] != "—") | (ex["raw"] != ex["clean"])].copy()
if len(cand) < 3:
    cand = ex.copy()
cand = cand.head(5).copy


In [18]:
# Salvar e sumário

# Grava o CSV completo (mesmo schema + RESUMO_PREP_BERTOPIC)
df.to_csv(OUT_CSV, index=False, encoding="utf-8")

# Confirmação
ok_col = "RESUMO_PREP_BERTOPIC" in df.columns
print("\n======== RESUMO ========")
print(f"Arquivo salvo em: {OUT_CSV.resolve()}")
print(f"Linhas: {len(df)}  |  Coluna RESUMO_PREP_BERTOPIC presente? {ok_col}")
print("Primeiros campos:", list(df.columns)[:12], "...")
display(df[["DOC_ID", "RESUMO_PREP_BERTOPIC"]].head(5))



======== RESUMO ========
Arquivo salvo em: C:\Users\User\Desktop\TCC\Notebooks locais\analise_topicos_tcc\data\interim\bertopic\prep.csv
Linhas: 423  |  Coluna RESUMO_PREP_BERTOPIC presente? True
Primeiros campos: ['DOC_ID', 'ano', 'titulo', 'autor', 'orientador', 'resumo', 'url', 'RESUMO_PREP_BERTOPIC'] ...


,DOC_ID,RESUMO_PREP_BERTOPIC
0,0,gerenciar a agenda e fornecer informações ao c...
1,1,"doenças oftalmológicas, como catarata, glaucom..."
2,2,o presente trabalho busca compreender se a uti...
3,3,este estudo investiga o impacto de ferramentas...
4,4,no cenário atual do desenvolvimento de softwar...
